In [1]:
from Dataload import dataload
import torch.nn.functional as F

import torch
import torch.nn as nn
from torchsummary import summary
from sklearn.metrics import confusion_matrix
from torch.utils.data import Dataset, DataLoader
import matplotlib.pylab as plt
from torchvision import  utils

from src import detect_faces, show_bboxes
from PIL import Image
import numpy as np
from module import video_model
from constant import EMOTIPATH
import os
import pandas as pd
import time
from tqdm.notebook import tqdm
import cv2
from torchvision import transforms


In [87]:

frame_data=torch.rand((24,3,256,256))
face_data=torch.rand((24,5,3,64,64))

In [88]:
frame_embedded_data=torch.rand((32,24,1000))

In [89]:
face_embedded_data=torch.rand((32,24,5,100))

In [90]:
class Video_modeller(nn.Module):
    ''' Scaled Dot-Product Attention '''

    def __init__(self,frame):
        super().__init__()
        
        self.en1=video_model.Encoder(num_frame=5,dim=100,model=None,isframe=0,label=256)
        self.en2=video_model.Encoder(num_frame=frame,dim=256,model=None,isframe=1,label=3)
        
        self.fc1=nn.Linear(1000,256)
        self.fc2=nn.Linear(512,256)
    def face_embedder(self,x,t,encoder):
        x1 = (encoder(x[:, t,:]))  # Pretrained_Densenet            
        x1 = x1.view(x1.size(0), -1)
        
        return x1

    def check_raw(self,x,dim):
        if x.shape[-1]==dim:
            return False
        return True 
        
        return None
    def stack_face_encoder(self,x):
        ##Transformation to: Frames*Channel*width*height
        cnn_embed_seq = []
     
        for t in range(x.size(1)):
            with torch.no_grad():
                x1=self.face_embedder(x,t,self.en1)
                cnn_embed_seq.append(x1)            
        cnn_embed_seq = torch.stack(cnn_embed_seq, dim=0).transpose_(0, 1)
        return cnn_embed_seq
    
    def forward(self,x,y):
        initial_time=time.time()
        if y is not None:
            start=time.time()
            frame_local=self.stack_face_encoder(y)
            end=time.time()
            print(end-start)
        if x is not None:
            frame_global=self.fc1(x)
        if y is None:
            return self.en2(frame_global)
        if x is None:
            return self.en2(frame_local)            
            
        
        frames_embedding=torch.cat((frame_global,frame_local),2)
        frames_embedding=self.fc2(frames_embedding)
        vide_embed=self.en2(frames_embedding) 
        finish_time=time.time()
        print(finish_time-initial_time)
        return vide_embed
        

In [91]:
debug=Video_modeller(24)

In [93]:
debug(frame_embedded_data,face_embedded_data)

0.04388427734375
0.05385899543762207


tensor([[0.3142, 0.3443, 0.3415],
        [0.3145, 0.3448, 0.3408],
        [0.3153, 0.3425, 0.3421],
        [0.3143, 0.3434, 0.3423],
        [0.3152, 0.3411, 0.3437],
        [0.3158, 0.3433, 0.3409],
        [0.3126, 0.3436, 0.3438],
        [0.3139, 0.3448, 0.3413],
        [0.3135, 0.3442, 0.3423],
        [0.3160, 0.3432, 0.3408],
        [0.3145, 0.3433, 0.3422],
        [0.3160, 0.3427, 0.3413],
        [0.3124, 0.3459, 0.3417],
        [0.3151, 0.3447, 0.3402],
        [0.3154, 0.3432, 0.3414],
        [0.3135, 0.3433, 0.3432],
        [0.3148, 0.3431, 0.3421],
        [0.3137, 0.3454, 0.3409],
        [0.3141, 0.3438, 0.3421],
        [0.3148, 0.3439, 0.3413],
        [0.3127, 0.3455, 0.3418],
        [0.3140, 0.3435, 0.3425],
        [0.3129, 0.3453, 0.3419],
        [0.3129, 0.3440, 0.3431],
        [0.3154, 0.3441, 0.3404],
        [0.3153, 0.3436, 0.3411],
        [0.3153, 0.3419, 0.3428],
        [0.3146, 0.3446, 0.3408],
        [0.3136, 0.3443, 0.3421],
        [0.314

In [66]:
test=video_model.Encoder(num_frame=24,dim=512,model=None,isframe=0,label=3)

In [22]:
test(torch.rand((1,24,512))).shape

torch.Size([1, 3])

In [18]:
Train_label=os.path.join(EMOTIPATH,"Train_labels.txt")
Train_video=os.path.join(EMOTIPATH,"Train")
frame_num=24
train_data=Raw_Data(Train_label,base_path_v=Train_video)


NameError: name 'Raw_Data' is not defined

In [ ]:
class Raw_Data():
    def __init__(self,csv_file,base_path_v=None,):
        
        self.max_frame_num=24
        self._table = pd.read_csv(csv_file,delimiter=' ')

        self._base_path_v=base_path_v
  

            

        self.transform=transforms.Compose([
                     transforms.Resize((256,256)),
                     transforms.ToTensor(),   
                     transforms.Normalize((0.5,0.5,0.5 ), (0.5, 0.5,0.5))])
    def __len__(self):
        return len(self._table)

    def __getitem__(self, idx):
        folder_name = os.path.join(self._base_path_v,self._table.Vid_name[idx])
        frame_list=os.listdir(folder_name)


        return folder_name,frame_list

In [ ]:
train_data[566]

In [ ]:
from operator import itemgetter, attrgetter

In [ ]:
def extract_best(num_frame,data):

for index,(base_path,imglist) in enumerate(train_data):
    confidence_score_list=[]
    for img_frame in imglist:
        
        img=Image.open(os.path.join(base_path,img_frame))
        total_confidence_score=sum(detect_faces(img)[0][0:5,-1])
        confidence_score_list.append((total_confidence_score,img_frame))
    confidence_score_list=sorted(confidence_score_list, key=itemgetter(0),reverse=True)
    sorted(confidence_score_list[0:num_frame]
    break


In [ ]:
confidence_score_list[0:5]

In [ ]:
confidence_score_list=sorted(confidence_score_list, key=itemgetter(0),reverse=True)

In [ ]:
confidence_score_list[0:5]

In [ ]:
sorted(confidence_score_list[0:5],key=itemgetter(0))

In [ ]:
img=Image.open(os.path.join(train_data[0][0],train_data[0][1][1]))

In [ ]:
img

In [ ]:
confidence_score=sum(detect_faces(img)[0][0:5,-1])

In [ ]:
sum(confidence_score)

In [ ]:
faces[0][0:5,-1]

In [ ]:
    
class cv_data(Dataset):
    def __init__(self,csv_file,sub_csv_file=None,
                 base_path_v=None,base_path_a=None,frame_num=16,strict_name=True,name_format=9,embedding=False):
        
        self.max_frame_num=24
        self._table = pd.read_csv(csv_file,delimiter=' ')
        if sub_csv_file is None:
            self._table_embedding=None
        else:
            self._table_embedding=pd.read_csv(sub_csv_file)
        self.frame_num = frame_num
        self._base_path_v=base_path_v
        self._base_path_a=base_path_a
        self.embedding=embedding
        if strict_name:
            self.name_format=name_format
            

        self.transform=transforms.Compose([
                     transforms.ToTensor(),   
                     transforms.Normalize((0.5,0.5,0.5 ), (0.5, 0.5,0.5))])
    def __len__(self):
        return len(self._table)

    def __getitem__(self, idx):
        folder_name = os.path.join(self._base_path_v,self._table.Vid_name[idx])
        first=True
        audio_img=np.zeros((1,1))
        labels = torch.from_numpy(np.array(self._table.Label[idx]))-1
            
        if self._table_embedding is not None:
            temp_frame_embedding=torch.from_numpy(np.array(self._table_embedding.Embedding[idx].split(),dtype=float)).reshape((self.max_frame_num,-1))
            frame_data=torch.empty(size=(self.frame_num,1000),dtype=torch.double)
            if self.frame_num<25:
                index=np.linspace(0,23,self.frame_num,dtype=int)
 
                for i,copy in enumerate(index):
                    frame_data[i]=temp_frame_embedding[copy]
            return (frame_data,audio_img,labels)
    

            
        frame_raw_list=os.listdir(folder_name)
        frame_len=len(frame_raw_list)

        frame_raw_list=sorted(frame_raw_list)

        frame_list=[]
        if frame_len<self.frame_num:
            for index_0 in range(frame_len):
                frame_path=os.path.join(folder_name,frame_raw_list[index_0])
                
                tempimg=cv2.imread(frame_path,cv2.IMREAD_COLOR)  
                tempimg=cv2.resize(tempimg, (256,256), interpolation = cv2.INTER_AREA)
               # frame_class=frame(tempimg,frame_path)
                frame_list.append(self.transform(tempimg))
        else:    
            frame_index=(np.linspace(0,frame_len-1,self.frame_num,dtype=int))

            for index_2 in frame_index:
                frame_path=os.path.join(folder_name,frame_raw_list[index_2])
                tempimg=cv2.imread(frame_path,cv2.IMREAD_COLOR)
                tempimg=cv2.resize(tempimg, (256,256), interpolation = cv2.INTER_AREA)

                #frame_class=frame(tempimg,frame_path)
                frame_list.append(self.transform(tempimg))
        frame_data=torch.stack(frame_list,dim=0)
        return (frame_data,audio_img,labels)

In [ ]:
    
class pil_data(Dataset):
    def __init__(self,csv_file,sub_csv_file=None,
                 base_path_v=None,base_path_a=None,frame_num=16,strict_name=True,name_format=9,embedding=False):
        
        self.max_frame_num=24
        self._table = pd.read_csv(csv_file,delimiter=' ')
        if sub_csv_file is None:
            self._table_embedding=None
        else:
            self._table_embedding=pd.read_csv(sub_csv_file)
        self.frame_num = frame_num
        self._base_path_v=base_path_v
        self._base_path_a=base_path_a
        self.embedding=embedding
        if strict_name:
            self.name_format=name_format
            

        self.transform=transforms.Compose([
                     transforms.Resize((256,256)),
                     transforms.ToTensor(),   
                     transforms.Normalize((0.5,0.5,0.5 ), (0.5, 0.5,0.5))])
    def __len__(self):
        return len(self._table)

    def __getitem__(self, idx):
        folder_name = os.path.join(self._base_path_v,self._table.Vid_name[idx])
        first=True
        audio_img=np.zeros((1,1))
        labels = torch.from_numpy(np.array(self._table.Label[idx]))-1
            
        if self._table_embedding is not None:
            temp_frame_embedding=torch.from_numpy(np.array(self._table_embedding.Embedding[idx].split(),dtype=float)).reshape((self.max_frame_num,-1))
            frame_data=torch.empty(size=(self.frame_num,1000),dtype=torch.double)
            if self.frame_num<25:
                index=np.linspace(0,23,self.frame_num,dtype=int)
 
                for i,copy in enumerate(index):
                    frame_data[i]=temp_frame_embedding[copy]
            return (frame_data,audio_img,labels)
    
        
            
        frame_raw_list=os.listdir(folder_name)
        frame_len=len(frame_raw_list)

        frame_raw_list=sorted(frame_raw_list)

        frame_list=[]
        if frame_len<self.frame_num:
            for index_0 in range(frame_len):
                frame_path=os.path.join(folder_name,frame_raw_list[index_0])
                
                tempimg=Image.open(frame_path)       
               # frame_class=frame(tempimg,frame_path)
                frame_list.append(self.transform(tempimg))
        else:    
            frame_index=(np.linspace(0,frame_len-1,self.frame_num,dtype=int))

            for index_2 in frame_index:
                frame_path=os.path.join(folder_name,frame_raw_list[index_2])
                tempimg=Image.open(frame_path)
                #frame_class=frame(tempimg,frame_path)
                frame_list.append(self.transform(tempimg))
        frame_data=torch.stack(frame_list,dim=0)
        return (frame_data,audio_img,labels)

In [ ]:
Train_label=os.path.join(EMOTIPATH,"Train_labels.txt")
Train_video=os.path.join(EMOTIPATH,"Train")
Val_labels=os.path.join(EMOTIPATH,"Val_labels.txt")
Val_video=os.path.join(EMOTIPATH,"Val")
train_data_pil=pil_data(Train_label,base_path_v=Train_video,frame_num=24)
train_dataloader_pil = DataLoader(train_data_pil, batch_size=24
                       , num_workers=0,shuffle=False)                    
                    

In [ ]:
train_data_cv=cv_data(Train_label,base_path_v=Train_video,frame_num=24)
train_dataloader_cv = DataLoader(train_data_cv, batch_size=24
                       , num_workers=0,shuffle=False)       

In [ ]:
start=time.time()
for i in enumerate(train_dataloader_pil):
    break

end=time.time()
print(end-start)

In [ ]:
start=time.time()
for i in enumerate(train_dataloader_cv):
    break

end=time.time()
print(end-start)

In [ ]:
class Frame_Face_Data(Dataset):
    def __init__(self,csv_file,sub_csv_file=None,base_path_v=None):
        
        self.max_frame_num=5
        self._table = pd.read_csv(csv_file,delimiter=' ')
        self._table_embedding=None
        self.transform=transforms.Compose([
                 #    transforms.Resize((256,256)),
                     transforms.ToTensor(),   
                    transforms.Normalize((0.5,0.5,0.5 ), (0.5, 0.5,0.5))
        ])
        self.endPad=self.transform(Image.new(mode='RGB', size=(64,64), color=0))
        self._base_path_v=base_path_v
        
    def __len__(self):
        return len(self._table)
    def _stacking_face(self,face_num,faces):

        face_v=torch.zeros((face_num,3,64,64))
        for i in range(face_num):
            face_v[i]=faces[:,:,i*64:i*64+64]
        return face_v

    def __getitem__(self, idx):
        folder_name = os.path.join(self._base_path_v,self._table.Vid_name[idx])

        labels = torch.from_numpy(np.array(self._table.Label[idx]))-1
            
        if self._table_embedding is not None:
            temp_frame_embedding=torch.from_numpy(np.array(self._table_embedding.Embedding[idx].split(),dtype=float)).reshape((self.max_frame_num,-1))
            frame_data=torch.empty(size=(self.frame_num,1000),dtype=torch.double)
            if self.frame_num<25:
                index=np.linspace(0,23,self.frame_num,dtype=int)
 
                for i,copy in enumerate(index):
                    frame_data[i]=temp_frame_embedding[copy]
            return (frame_data,audio_img,labels)
    

        face_len=5
        face_raw_list=os.listdir(folder_name)
        face_raw_list=sorted(face_raw_list)

        face_list=[]
        for index_0 in range(face_len):
                face_path=os.path.join(folder_name,face_raw_list[index_0])               
                tempimg=Image.open(face_path)       
                face_list.append(self._stacking_face(face_len,self.transform(tempimg)))
            

        face_data=torch.stack(face_list,dim=0)
            
        return (face_data)

In [ ]:
Train_label=os.path.join(EMOTIPATH,"Train_labels.txt")
Train_face=os.path.join(EMOTIPATH,"Face_Cropped","Train")
Val_labels=os.path.join(EMOTIPATH,"Val_labels.txt")
Val_face=os.path.join(EMOTIPATH,"Face_Cropped","Val")

In [ ]:
test=Frame_Face_Data(Train_label,Train_face,base_path_v=Train_face)

In [ ]:
test[0].shape

In [ ]:
trans = transforms.ToPILImage()

In [ ]:
data=test[0][0]*0.5+0.5

In [ ]:
data.shape

In [ ]:
def stacking_face(face_num,faces):

    face_v=torch.zeros((face_num,3,64,64))
    for i in range(face_num):
        face_v[i]=faces[:,:,i*64:i*64+64]
    return face_v


In [ ]:
for i in range(5):
    
    plt.subplot(5,1,i+1)
    plt.imshow((trans((face_v[i]).squeeze(0))))

In [ ]:
face_v[4].shape